In [2]:
# H -> Helix
# T -> Turn
# S -> Strands


palpha = {
    'Glu': 1.53,
    'Ala': 1.45,
    'Leu': 1.34,
    'His': 1.24,
    'Met': 1.20,
    'Gln': 1.17,
    'Trp': 1.14,
    'Val': 1.14,
    'Phe': 1.12,
    'Lys': 1.07,
    'Ile': 1.00,
    'Asp': 0.98,
    'Thr': 0.82,
    'Ser': 0.79,
    'Arg': 0.79,
    'Cys': 0.77,
    'Asn': 0.73,
    'Tyr': 0.61,
    'Pro': 0.59,
    'Gly': 0.53
}

pbeta = {
    'Met': 1.67,
    'Val': 1.65,
    'Ile': 1.60,
    'Cys': 1.30,
    'Tyr': 1.29,
    'Phe': 1.28,
    'Gln': 1.23,
    'Leu': 1.22,
    'Thr': 1.20,
    'Trp': 1.19,
    'Ala': 0.97,
    'Arg': 0.90,
    'Gly': 0.81,
    'Asp': 0.80,
    'Lys': 0.74,
    'Ser': 0.72,
    'His': 0.71,
    'Asn': 0.65,
    'Pro': 0.62,
    'Glu': 0.26
}

aminoprefix = {
    'A': 'Ala',
    'R': 'Arg',
    'N': 'Asn',
    'D': 'Asp',
    'C': 'Cys',
    'Q': 'Gln',
    'E': 'Glu',
    'G': 'Gly',
    'H': 'His',
    'I': 'Ile',
    'L': 'Leu',
    'K': 'Lys',
    'M': 'Met',
    'F': 'Phe',
    'P': 'Pro',
    'S': 'Ser',
    'T': 'Thr',
    'W': 'Trp',
    'Y': 'Tyr',
    'V': 'Val',
    'B': 'Asx',
    'Z': 'Glx',
}


# a b c d e f
# 0 1 2 3 4 5

# len = 6, i+6 <= len(string)

# i:i+6 abcdef len = 6 string[7] =

# AFPS
# Ala-Phe-Pro-Ser
# 1.45+1.12+0.59+0.79

# extends the string from the rhs

def extendrightalpha(i, string):
    extndregion = ''
    ptrlst = i + 5  # ptr to last element
    while i + 6 < len(string):  # exit cond
        short = string[i + 3] + string[i + 4] + string[i + 5] + string[i + 6]  # extract substr
        scoreshort = 0  # find score
        for j in short:
            threelettername = aminoprefix.get(j)
            scoreshort = scoreshort + palpha.get(threelettername)
        if scoreshort < 4:  # if score < 4 break
            break
        else:
            extndregion = extndregion + string[i + 6]  # else add to our region
            ptrlst = i + 6  # update ptr

        i = i + 1

    return extndregion, ptrlst  # return extd,and ptr to last


def extendrightbeta(i, string):
    extndregion = ''
    ptrlst = i + 4  # ptr to last
    while i + 5 < len(string):  # exit cond
        short = string[i + 2] + string[i + 3] + string[i + 4] + string[i + 5]  # extract subs
        scoreshort = 0  # init score with 0
        for j in short:
            threelettername = aminoprefix.get(j)
            scoreshort = scoreshort + pbeta.get(threelettername)
        if scoreshort < 4:  # if score is less than 4
            break
        else:
            extndregion = extndregion + string[i + 5]  # add the char
            ptrlst = i + 5  # update the ptr

        i = i + 1

    return extndregion, ptrlst  # return regionexd, ptr to last


def extendleftbeta(i, string):
    extndregion = ''
    leftptr = i  # ptr to left
    while i - 1 >= 0:  # exit cond
        short = string[i - 1:i + 3]  # get substr
        scoreshort = 0  # init score with 0
        for j in short:
            threelettername = aminoprefix.get(j)
            scoreshort = scoreshort + pbeta.get(threelettername)
        if scoreshort < 4:  # if score is less than 4
            break
        else:
            extndregion = extndregion + string[i - 1]  # add to extd region
            leftptr = i - 1  # update the ptr
        i = i - 1

    return extndregion[::-1], leftptr  # return rev extd and left ptr


# same as above function

def extendleftalpha(i, string):
    extndregion = ''
    leftptr = i
    while i - 1 >= 0:
        short = string[i - 1:i + 3]
        scoreshort = 0
        for j in short:
            threelettername = aminoprefix.get(j)
            scoreshort = scoreshort + palpha.get(threelettername)
        if scoreshort < 4:
            break
        else:
            extndregion = extndregion + string[i - 1]
            leftptr = i - 1
        i = i - 1

    return extndregion[::-1], leftptr


def nucleationsitesalpha(string):
    i = 0  # set inital ptr to 0th character
    nsitelst = []  # nucleation sites
    countlst = []  # checking for how many elements P(H) >= 1
    goodlst = []  # extd lst are stored here
    tuplelst = []  # indexes are stored
    while i <= len(string) - 6:  # when we don't have chars for window left exit
        region = string[i:i + 6]  # extract substr
        count = 0
        for j in region:
            threelettername = aminoprefix.get(j)
            if palpha.get(threelettername) >= 1:
                count = count + 1
        nsitelst.append(region)  # update the lst after processed
        countlst.append(count)  # update count for that region
        if count >= 4:
            extr, rightptr = extendrightalpha(i, string)  # check left
            extl, leftptr = extendleftalpha(i, string)  # check right
            goodlst.append(extl + region + extr)  # add to lst
            tuplelst.append((leftptr, rightptr))  # add the ptr to lst
        i = i + 1
    # print(tuplelst)
    lst = []  # if nothing then we leave it blank
    for o in string:
        lst.append('-')
    # update pos suitable for helix now
    for p, tup in enumerate(tuplelst):
        start = tup[0]
        end = tup[1]
        for x in range(start, end + 1):
            lst[x] = "H"

    aminoacidstring = ''.join(lst)
    #print(aminoacidstring)
    return aminoacidstring


def nucleationsitesbeta(string):
    i = 0  # set inital ptr to 0th character
    nsitelst = []  # nucleation sites
    countlst = []  # checking for how many elements P(H) >= 1
    goodlst = []  # extd regions with inital are stored here
    tuplelst = []  # ptr to strt and end
    while i <= len(string) - 5:  # just the change is window size
        region = string[i:i + 5]
        count = 0
        for j in region:
            threelettername = aminoprefix.get(j)
            if pbeta.get(threelettername) >= 1:
                count = count + 1
        nsitelst.append(region)  # update the lst after processed
        countlst.append(count)  # update count for that region
        if count >= 3:
            extr, rightptr = extendrightbeta(i, string)  # check left
            extl, leftptr = extendleftbeta(i, string)  # check right
            goodlst.append(extl + region + extr)  # add to lst
            tuplelst.append((leftptr, rightptr))  # add index to tuplelst
        i = i + 1
    # print(goodlst)
    lst = []  # if nothing than leave it blank
    for o in string:
        lst.append('-')
    # else there is an S for positions which have them
    for p, tup in enumerate(tuplelst):
        start = tup[0]
        end = tup[1]
        for x in range(start, end + 1):
            lst[x] = "S"

    aminoacidstring = ''.join(lst)
    #print(aminoacidstring)
    return aminoacidstring


def finalmerge(s1, s2, string):
    mylst = []  # chars coming per position maybe 0,1,2
    conflictreg = []  # tuple for strt and end of conficting regions
    scores = []
    for p in string:
        mylst.append(0)
    for i in range(len(string)):
        if s1[i] == 'H':
            mylst[i] = mylst[i] + 1
        if s2[i] == 'S':
            mylst[i] = mylst[i] + 1
        scores.append(str(mylst[i]))
    j = 0
    # print(''.join(scores))
    # now what the while loop does is it finds a start point and keeps on going till it finds an end point, then after it finds the
    # end point, it sets the start pointer at the same position as end pointer. exit condition is when whole string is iterated
    bigflag = True
    while j < len(string) and bigflag:
        strtptr = -1
        flag = False
        if mylst[j] == 2:
            strtptr = j
            temp = j
            flag = True
            for h in range(strtptr, len(string)):
                if h == len(string) - 1:
                    if mylst[h] == 2:
                        conflictreg.append((strtptr, h))
                        bigflag = False
                if mylst[h] == 1 or mylst[h] == 0:
                    conflictreg.append((strtptr, h - 1))
                    temp = h
                    flag = True
                    break
            j = temp
        if not flag:
            j = j + 1
    # print(conflictreg)
    # initally put blanks
    lst = []
    for o in string:
        lst.append('-')
    # for conficting regions, decide which one is better on basis of the dict of propencities
    for p, tup in enumerate(conflictreg):
        start = tup[0]
        end = tup[1]
        salpha = 0
        sbeta = 0
        for x in range(start, end + 1):
            salpha = salpha + palpha.get(aminoprefix.get(string[x]))
            sbeta = sbeta + pbeta.get(aminoprefix.get(string[x]))
        for y in range(start, end + 1):
            if salpha > sbeta:
                lst[y] = 'H'
            else:
                lst[y] = 'S'
    # not conficting regions which do not have 'H' or 'S' will have a 'T'
    for q in range(len(string)):
        if lst[q] == '-':
            if s1[q] == 'H':
                lst[q] = 'H'
            elif s2[q] == 'S':
                lst[q] = 'S'
            else:
                lst[q] = 'T'
    # print the final output as a string
    print(''.join(lst))
    return ''.join(lst)


string = "SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDTVYCPRHVICTAEDMLNPNYEDLLIRKSNHSFLVQAGNVQLRVIGHSMQNCLLRLKVDTSNPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNHTIKGSFLNGSCGSVGF"
print('Initial string given to us: ')
print(string)
print('Output: ')

myoutput = finalmerge(nucleationsitesalpha(string), nucleationsitesbeta(string), string)
# upload the pdb file and from the table, extract the col for string and then

print()
print('Now doing the 2nd part of the assignment')

strideoutput = "TTTTCCCCCHHHHHHCEEEEEETTEEEEEEEETTEEEEEGGGGCCHHHHHCCCHHHHHHHCCGGGCEEEETTEEECEEEEEEETTEEEEEECCCTTTTCCCCCCCCTTTEEEEEEEEETTEEEEEEEEEETTTTCBCCCCTTTTTTTEEC"
print('The output I got from the shride web-server: -- {Please read comments after this line} ')
print(strideoutput)
lstmyoutput = list(myoutput)
lststrideoutput = list(strideoutput)
print('X represents dis-similar region from stride and our chou fasman output')
# replace
# E->S E is for strand in web-server that's why
# to get this lstshrideoutput as shown below

# H -> Helix
# T -> Turn
# S -> Strands


# 'X' -> not matching region
#  character that matched for -> matching region
for i in range(len(string)):
    if lststrideoutput[i] == 'E':
        lststrideoutput[i] = 'S'
    else:
        lststrideoutput[i] = strideoutput[i]

checklst = []
dissim = []
# marking region of mismatch as X
match = 0
dis = ''
for i in range(len(string)):
    if lststrideoutput[i] == lstmyoutput[i]:
        match = match + 1
        checklst.append(lststrideoutput[i])
    else:
        checklst.append('X')


for i in range(len(checklst)):
    if checklst[i] == 'X':
        dis = dis + string[i]
    else:
        dis = dis + '-'



checklst = ''.join(checklst)



print(checklst)
print('By analysing we get: ')
print('Length of the strings = ', len(string))
print('Number of matches = ',match)
print('Number of mismatches = ',len(string)-match)
print('% accuracy wrt stride(web-server) output = ',match/len(string)*100)
print('If you want to see as a part of string that is dis-similar here it is')
print(dis)
print()
print('2B) The output we got from the web-server is more accurate as it also taken into account the hydrogen bond \n'
      'energy(HBE) and torsional angle(it means it also thinks of the backbone geometry) and gives us the optimized \n'
      'result that suits the series of amino acid given to us(protein seq). Whereas in Chou-Fasman method, '
      'someone specific \n'
      'probability parameters have been defined for each amino acid in its 2ndary structure using it we are predicting '
      'structure \n'
      'of our protein.  The difference also lies where our web-server based method also predicts coils, bridges and \n'
      '310 helices which obvs our chou fasman does not, also it has less efficiency from the web-server method as \n'
      'in my statistics. Long range interactions are neglected in chou fasman method it is based on propensity per \n'
      'amino acid. Web-server is 80% accurate btw. If I told to print the part of the string for which the output was '
      'different it comes out: ')
print(dis)
print('So the question arises why is it different for these particular regions only , one of the reasons could be '
      'interactions of this part of \n'
      'protein with other part of the protein leading to different crystal structure in real world. Or maybe '
      'unexpected folding to attain \n'
      'low entropy and attaining stability. Ability to form hydrogen bonds to attain stability, electrostatic forces '
      'of attraction \n'
      'vader weal forces and the ability to free rotation across a sigma bond whereas restricted rotation across pie bonds.')

Initial string given to us: 
SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDTVYCPRHVICTAEDMLNPNYEDLLIRKSNHSFLVQAGNVQLRVIGHSMQNCLLRLKVDTSNPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNHTIKGSFLNGSCGSVGF
Output: 
THHHHHHHHHHHSSSSSSSSSSSSSSHHHHHHHHHSSSSHHHHHHHHHHHHTTHHHHHHHHHHSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSTTTTHSSSSSSSSSSSSSSSSSSSSSTTSSSSSSSSSSTTTHHHHHHTTTTTTTTT

Now doing the 2nd part of the assignment
The output I got from the shride web-server: -- {Please read comments after this line} 
TTTTCCCCCHHHHHHCEEEEEETTEEEEEEEETTEEEEEGGGGCCHHHHHCCCHHHHHHHCCGGGCEEEETTEEECEEEEEEETTEEEEEECCCTTTTCCCCCCCCTTTEEEEEEEEETTEEEEEEEEEETTTTCBCCCCTTTTTTTEEC
X represents dis-similar region from stride and our chou fasman output
TXXXXXXXXHHHXXXXSSSSSSXXSSXXXXXXXXXSSSSXXXXXXHHHHHXXXHHHHHHHXXXXXXSSSSXXSSSXSSSSSSSXXSSSSSSXXXTTTTXXXXXXXXXXXSSSSSSSSSXXXXSSSSSSSSXXTTXXXXXXXTTTTTTXXX
By analysing we get: 
Length of the strings =  150
Number of matches =  77
Number of mismatches =  73
% accuracy wrt stride(web-server) output =  51.33333333